In [ ]:
import astropy
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib.gridspec as gridspec
from astropy import coordinates as coord
from astropy import units as u
from astropy.io import ascii, fits
from astropy.nddata import Cutout2D
from astropy import wcs
import FITS_tools
from astropy.wcs import WCS
from astropy.units import cds

import aplpy

# Package versions:
print('Running on:')
print(f'\tAstropy\t\t{astropy.__version__}')
print(f'\tMatplotlib\t{plt.matplotlib.__version__}')
print(f'\tNumpy\t\t{np.__version__}')
print(f'\tAplpy\t\t{aplpy.__version__}')
#
%matplotlib inline
#python3.6
rc('text', usetex=True)
font = {'family' : 'serif','size'   : 18}
rc('font', **font)

In [ ]:
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)

In [ ]:
def fix_aplpy_fits(aplpy_obj, dropaxis=2):
    """This removes the degenerated dimensions because in APLpy 2.X they fucked it up...
    The input must be the figure returned by aplpy.FITSFigure().
    `dropaxis` is the index where to start dropping the axis (by default it assumes the 3rd,4th place).
    """
    temp_wcs = aplpy_obj._wcs.dropaxis(2)
    temp_wcs = temp_wcs.dropaxis(2)
    aplpy_obj._wcs = temp_wcs

In [ ]:
def contour_levels(rms, nsigma=3, n_pos=5, n_neg=1):
    """ Return a list with the values of fluxes to plot a contour figure.
    n_* are the number of contours to positive values (above rms level) or
    negative ones (below rms level).
    It returns three lists:
    - with the negative and positive values for contours.
    - with only the positive contour values.
    - with only the negative contour values"""
    conts_p = [ rms*nsigma*2**(i/2.) for i in range(n_pos)]
    conts_n = [ -rms*5*2**(i/2.) for i in range(n_neg)]
    conts = conts_n[::-1]
    [conts.append(c) for c in conts_p]
    return conts, conts_p,conts_n 

In [ ]:
# Because Difmap FITS files do not insert correctly BMAJ, BMIN, PA in the header..
bursts = {'B1': {'beam': {'major': 4*u.mas, 'minor': 2*u.mas, 'angle': 16}},
          'B2': {'beam': {'major': 10*u.mas, 'minor': 5*u.mas, 'angle': 15}}
          }    

In [ ]:
#Adding folder to the path
folder = '14arcsec'
fov = '55arcsec'
#load data
L_band = folder + '/L-band.fits'
S_band = folder + '/S-band_conv.fits'

# First: create cutouts with updated WCS with the same centroid position

In [ ]:
centroid_position = coord.SkyCoord('19h58m14.81720s', '35d16m52.151626s', frame='icrs')
cutout_size = u.Quantity((55., 55.), u.arcmin) #edit based on the size of the spectral index map you want to make
print(centroid_position.ra.deg, centroid_position.dec.deg)

In [ ]:
filename = L_band
hdu = fits.open(filename)[0]
data = hdu.data[0,0,:,:] #modify data to change the fits file into a 2D shape
hdu.header['NAXIS'] = 2
hdu.header['WCSAXES']=2
# delete all keywords of additional axis
del hdu.header['NAXIS3']
del hdu.header['NAXIS4']
del hdu.header['CTYPE3']
del hdu.header['CRPIX3']
del hdu.header['CRVAL3']
del hdu.header['CDELT3']
del hdu.header['CUNIT3']
del hdu.header['CTYPE4']
del hdu.header['CRPIX4']
del hdu.header['CRVAL4']
del hdu.header['CDELT4']
del hdu.header['CUNIT4']
wcs = WCS(hdu.header)
#print(hdu.header)
# Make the cutout, including the WCS
cutout = Cutout2D(data, position=centroid_position, size=cutout_size, wcs=wcs)

# Put the cutout image in the FITS HDU
hdu.data = cutout.data

# Update the FITS header with the cutout WCS
hdu.header.update(cutout.wcs.to_header())

# Write the cutout to a new FITS file
cutout_filename = folder + '/L_band_cropped_'+fov+'.fits'
hdu.writeto(cutout_filename, overwrite=True)

In [ ]:
# Load the image and the WCS: S band
filename = S_band
hdu = fits.open(filename)[0]
data = hdu.data[0,0,:,:] #modify data to change the fits file into a 2D shape
hdu.header['NAXIS'] = 2
hdu.header['WCSAXES']=2
# delete all keywords of additional axis
del hdu.header['NAXIS3']
del hdu.header['NAXIS4']
del hdu.header['CTYPE3']
del hdu.header['CRPIX3']
del hdu.header['CRVAL3']
del hdu.header['CDELT3']
del hdu.header['CUNIT3']
del hdu.header['CTYPE4']
del hdu.header['CRPIX4']
del hdu.header['CRVAL4']
del hdu.header['CDELT4']
del hdu.header['CUNIT4']
wcs = WCS(hdu.header)

# Make the cutout, including the WCS
cutout = Cutout2D(data, position=centroid_position, size=cutout_size, wcs=wcs)

# Put the cutout image in the FITS HDU
hdu.data = cutout.data

# Update the FITS header with the cutout WCS
hdu.header.update(cutout.wcs.to_header())

# Write the cutout to a new FITS file
cutout_filename = folder + '/S_band_cropped_'+fov+'.fits'
hdu.writeto(cutout_filename, overwrite=True)

In [ ]:
Meer_S = folder + '/S_band_cropped_'+fov+'.fits'
Meer_L = folder + '/L_band_cropped_'+fov+'.fits'

In [ ]:
#Load data into arrays Sband
fh = fits.open(Meer_S)
data_S = fh[0].data.squeeze() # drops the size-1 axes if present
header_S = fh[0].header
mywcs_S = WCS(header_S).celestial

fh.close()

In [ ]:
fh = fits.open(Meer_L)
data_L = fh[0].data.squeeze() # drops the size-1 axes if present
header_L = fh[0].header
mywcs_L = WCS(header_L).celestial

fh.close()

In [ ]:
# Frequencies in GHz:
nu_L = 1.2840
nu_S = 2.6249

In [ ]:
# RMS values (need to check what units these are in):
dflux_L = 24e-6 #Jy/beam here    
dflux_S = 10e-6

In [ ]:
alpha = np.zeros((len(data_L), len(data_L)), dtype='float32')
dalpha = np.zeros((len(data_L), len(data_L)), dtype='float32')
for i in range(len(data_L)):
    for j in range(len(data_L)):
        
        flux_L = data_L[i,j]

        # Note: this is a correction since the two datasets have the same beam size, but not the same pixel size.
        # So the pixel index needs to be re-scaled for the MeerKAT S-band data.
        i_S = int(i * len(data_S) / len(data_L))
        j_S = int(j * len(data_S) / len(data_L))
        
        flux_S = data_S[i_S, j_S]

        # Only calculating alpha when the flux density in both bands exceeds a threshold:
        if flux_L > 2*dflux_L and flux_S > 2*dflux_S:
            
            alpha_ij = np.log10(flux_L / flux_S) / np.log10(nu_L / nu_S)
            dalpha_ij = abs((1. / np.log10(nu_L / nu_S)) * np.sqrt((dflux_L/flux_L)**2. + (dflux_S/flux_S)**2.))
            print(alpha_ij,dalpha_ij)

        # Otherwise, save a NaN so that nothing gets plotted for those pixels. 
        else:
                
            alpha_ij = np.NaN
            dalpha_ij = np.NaN
            
        alpha[i,j] = alpha_ij       
        dalpha[i,j] = dalpha_ij   
print(len(alpha),len(data_L))   

In [ ]:
# Opening the L-band image and copying its header and wcs for the new alpha file (since it will have the same)
fh = fits.open(Meer_L)
header_ALPHA = fh[0].header
mywcs_ALPHA = WCS(header_ALPHA).celestial
# Setting the data equal to the calculated spectral indices:
data_ALPHA = alpha

new_header_ALPHA = FITS_tools.strip_headers.flatten_header(header_ALPHA)
new_fh_ALPHA = fits.PrimaryHDU(data=data_ALPHA, header=new_header_ALPHA)

new_fh_ALPHA.writeto(folder+'/alpha'+fov+'.fits', overwrite=True)

fh.close()

In [ ]:
# Opening the L-band image and copying its header and wcs for the new alpha uncertainty file (since it will have the same)
fh = fits.open(Meer_L)
header_dALPHA = fh[0].header
mywcs_dALPHA = WCS(header_dALPHA).celestial

# Setting the data equal to the calculated spectral indices:
data_dALPHA = dalpha

new_header_dALPHA = FITS_tools.strip_headers.flatten_header(header_dALPHA)
new_fh_dALPHA = fits.PrimaryHDU(data=data_dALPHA, header=new_header_dALPHA)

new_fh_dALPHA.writeto(folder+'/dalpha'+fov+'.fits', overwrite=True)

fh.close()

In [ ]:
fig = aplpy.FITSFigure(folder+'/alpha'+fov+'.fits')
#fix_aplpy_fits(fig)
fig.recenter(centroid_position.ra.value, centroid_position.dec.value, radius=0.5)
fig.show_colorscale(cmap='jet', vmin=-3, vmax=1, stretch='linear') 
fig.add_colorbar()

# Set the font properties for the colorbar axis label
fig.colorbar.set_axis_label_text(r'Spectral index $\alpha$')
fig.colorbar.set_axis_label_font(size=18, weight='medium', 
                                 stretch='normal', family='serif', 
                                 style='normal', variant='normal')

# Set the font properties for the colorbar tick labels
fig.colorbar.set_font(size=18, weight='medium', 
                      stretch='normal', family='serif', 
                      style='normal', variant='normal')

fig.colorbar.set_location('top')
fig.axis_labels.set_font(**font)
fig.axis_labels.set_ypad(-0.4)
fig.axis_labels.set_xtext(r'R.A. (J2000)')
fig.axis_labels.set_ytext(r'Dec. (J2000)')
fig.add_beam()
fig.beam.set_color('w')
fig.beam.set_edgecolor('w')
fig.frame.set_color('k')
fig.ticks.set_color('k')
fig.tick_labels.set_font(**font)
#plt.show()
plt.savefig(folder+'/Spectral_map_'+fov+'.pdf', bbox_inches='tight')


In [ ]:
fig = aplpy.FITSFigure(folder+'/dalpha'+fov+'.fits')
#fix_aplpy_fits(fig)
fig.recenter(centroid_position.ra.value, centroid_position.dec.value, radius=0.5)
fig.show_colorscale(cmap='gnuplot_r',vmin=0,vmax=1.8, stretch='linear') 
fig.add_colorbar()

# Set the font properties for the colorbar axis label
fig.colorbar.set_axis_label_text(r'Uncertainty in $\alpha$')
fig.colorbar.set_axis_label_font(size=18, weight='medium', 
                                 stretch='normal', family='serif', 
                                 style='normal', variant='normal')

# Set the font properties for the colorbar tick labels
fig.colorbar.set_font(size=18, weight='medium', 
                      stretch='normal', family='serif', 
                      style='normal', variant='normal')

fig.colorbar.set_location('top')
fig.axis_labels.set_font(**font)
fig.axis_labels.set_ypad(-0.4)
fig.axis_labels.set_xtext(r'R.A. (J2000)')
fig.axis_labels.set_ytext(r'Dec. (J2000)')
fig.add_beam()
fig.beam.set_color('w')
fig.beam.set_edgecolor('w')
fig.frame.set_color('k')
fig.ticks.set_color('k')
fig.tick_labels.set_font(**font)
#plt.show()
plt.savefig(folder+'/Spectral_map_error_'+fov+'.pdf', bbox_inches='tight')

